<a href="https://colab.research.google.com/github/crlondono/prueba_repo_local/blob/master/prueba_tecnica_analitica_cumplimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content


In [ ]:
!ls

datos_con_id.csv  drive  sample_data


In [ ]:
# leemos los datos procesados en R
import pandas as pd
import numpy as np
import io

df = pd.read_csv('datos_con_id.csv', sep=';')

In [ ]:
df.head()

,id,frecuencia_pais,frecuencia_rem,riesgo_sucursal,frecuencia_suc,n_municipios_remesas,n_sucursales_remesas,n_paises_remesas,n_remesadoras,n_ordenantes_remesas,frecuencia_remsas,monto_remesas,riesgo_jurisdiccion_remesas,n_municipios_divisas,n_sucursales_divisas,frecuencia_divisas,monto_divisas,riesgo_remesadora_num
0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,788.96,1,0,0,0,0.0,2
1,2,34.0,33.0,1,34.0,1,1,1,2,2,34,8698.18,1,0,0,0,0.0,0
2,3,84.0,39.0,1,52.0,1,3,2,9,53,86,13726.62,2,0,0,0,0.0,2
3,4,3.0,3.0,1,3.0,1,1,1,1,1,3,161.71,1,0,0,0,0.0,0
4,5,6.0,5.0,1,11.0,1,1,2,3,9,11,11298.44,1,0,0,0,0.0,2


In [ ]:
# quitamos la variable id 
df_k = df.drop(['id'], axis=1)
df_k.head()

,frecuencia_pais,frecuencia_rem,riesgo_sucursal,frecuencia_suc,n_municipios_remesas,n_sucursales_remesas,n_paises_remesas,n_remesadoras,n_ordenantes_remesas,frecuencia_remsas,monto_remesas,riesgo_jurisdiccion_remesas,n_municipios_divisas,n_sucursales_divisas,frecuencia_divisas,monto_divisas,riesgo_remesadora_num
0,1.0,1.0,1,1.0,1,1,1,1,1,1,788.96,1,0,0,0,0.0,2
1,34.0,33.0,1,34.0,1,1,1,2,2,34,8698.18,1,0,0,0,0.0,0
2,84.0,39.0,1,52.0,1,3,2,9,53,86,13726.62,2,0,0,0,0.0,2
3,3.0,3.0,1,3.0,1,1,1,1,1,3,161.71,1,0,0,0,0.0,0
4,6.0,5.0,1,11.0,1,1,2,3,9,11,11298.44,1,0,0,0,0.0,2


In [ ]:
df_k.shape

(1643847, 17)

In [ ]:
# se escalan las variables
from sklearn.preprocessing import StandardScaler
df_sc = StandardScaler().fit_transform(df_k)

In [ ]:
df_sc
df_sc.shape

(1643847, 17)

In [ ]:
# importamos mini_batch_kmeans
from sklearn.cluster import MiniBatchKMeans

In [ ]:
l = []
for n in range(2,15):
  mb_kmeans = MiniBatchKMeans(n_clusters=n, n_init = 5, batch_size=50)
  mb_kmeans.fit(df_sc)
  tupla=(n, mb_kmeans.inertia_)
  l.append(tupla)

In [ ]:
# vamos las incercias dependiendo de la cantidad de grupos
l

[(2, 23440555.90393232),
 (3, 21311780.116020054),
 (4, 20426506.413302567),
 (5, 19554672.33707628),
 (6, 18703118.91066238),
 (7, 16988337.327483628),
 (8, 16532666.362632511),
 (9, 15340482.325785082),
 (10, 11437121.614829378),
 (11, 14498681.563530378),
 (12, 15037482.039980194),
 (13, 15195269.355400372),
 (14, 10321689.550846906)]

In [ ]:
# importamos el algoritmo k - medias
from sklearn.cluster import KMeans

In [ ]:
# vemos la variación de la incercia dependiendo del número de grupos
lista = []
for n in range(2,11):
  kmeans = KMeans(n_clusters=n)
  kmeans.fit(df_sc)
  tupla=(n, kmeans.inertia_)
  lista.append(tupla)

In [ ]:
# resultados lista de inercias por grupo
lista

[(2, 22932140.581028573),
 (3, 19305758.194083963),
 (4, 17428839.2112167),
 (5, 15966140.074335756),
 (6, 14614118.193463102),
 (7, 13303314.896972673),
 (8, 11995221.872379946),
 (9, 11198958.645253748),
 (10, 10420019.681830209)]

In [ ]:
# guardamos los modelos y las inercias para diferentes corridas de kmeans
kmeans_per_k = [KMeans(n_clusters=k, random_state=5).fit(df_sc)
                for k in range(2, 11)]
inertias = [model.inertia_ for model in kmeans_per_k]

In [ ]:
inertias

[22932139.91603025,
 19305755.546726808,
 17417511.8137562,
 15966140.074335756,
 14645119.098384839,
 13303281.28460783,
 11995226.4772325,
 11196049.153461281,
 10420021.379688416]

In [ ]:
kmeans_per_k

[KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
        random_state=5, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
        random_state=5, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
        random_state=5, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
        random_state=5, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
        random_state=5, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy

In [ ]:
# se calcula el porcentaje de registros por grupo
porcentajes_grupos = []
y_predichos = []

for km in kmeans_per_k:
  y_pred = km.fit_predict(df_sc)
  y_predichos.append(y_pred)
  count_arr = np.bincount(y_pred)
  porcentajes_grupos.append((len(count_arr), (count_arr/1643847)*100))
  

In [ ]:
porcentajes_grupos

[(2, array([88.41485856, 11.58514144])),
 (3, array([ 0.54116959, 87.91694118, 11.54188924])),
 (4, array([73.01050524,  3.93747107,  0.54110875, 22.51091495])),
 (5, array([72.23415561,  2.53551577, 15.54743233,  0.54110875,  9.14178753])),
 (6, array([70.98288344,  2.50193601,  1.90692929,  0.54110875, 15.09270632,
          8.97443619])),
 (7, array([5.40987087e-01, 7.16768653e+01, 2.53442078e+00, 9.08795040e+00,
         6.81085283e-01, 1.21665824e-04, 1.54785695e+01])),
 (8, array([7.07566458e+01, 2.42133240e+00, 1.88995691e+00, 5.40987087e-01,
         1.47892109e+01, 8.92169405e+00, 1.21665824e-04, 6.80051124e-01])),
 (9, array([1.87292370e+00, 6.88214292e+01, 7.58495164e+00, 5.40926254e-01,
         2.15889922e+00, 1.21665824e-04, 6.80111957e-01, 1.17362504e+01,
         6.60438593e+00])),
 (10, array([1.57983681e+01, 1.81318578e+00, 1.87626951e+00, 5.40926254e-01,
         1.01036775e+01, 5.86283273e+00, 1.21665824e-04, 6.80172790e-01,
         8.10008474e+00, 5.52243609e+01])

In [ ]:
from sklearn.metrics import silhouette_score
#n=2
silhouette_score(df_sc , kmeans_per_k[0].labels_, sample_size = 100000)

0.532996647798954

In [ ]:
# el score cambia muy poco al ampliar la muestra
silhouette_score(df_sc , kmeans_per_k[0].labels_, sample_size = 200000)

0.5329268975690012

In [ ]:
# n = 10
silhouette_score(df_sc , kmeans_per_k[8].labels_, sample_size = 100000)

0.3546064586481076

In [ ]:
# # el score cambia muy poco al ampliar la muestra
silhouette_score(df_sc , kmeans_per_k[8].labels_, sample_size = 200000)

0.354710880737771

In [ ]:
# se utiliza el silhouette_score para determinar el nro de grupos
from sklearn.metrics import silhouette_score 

lista_sil = []
for n in range(2,11):
  sil_score = silhouette_score(df_sc,kmeans_per_k[n].labels_, sample_size = 100000)
  lista_sil.append(sil_score)


In [ ]:
# teniendo en cuenta el silhouette score, las inercias y la cantidad de datos por grupo
# escogemos 3 como el número de grupos del agrupamiento
# se marcan entonces las marcas de cada grupo
df_resultado = df.assign(cluster = kmeans_per_k[1].labels_)

In [ ]:
# exportamos los resultados
df_resultado.shape

(1643847, 19)

In [ ]:
# tomamos sólo las columnas de id y cluster
df_resultado = df_resultado[['id','cluster']]

In [ ]:
df_resultado.to_csv('resultado_agrupamiento.csv', sep=',', index=False)